In [15]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
from tqdm import trange
import pandas as pd
pd.set_option('expand_frame_repr', False)
from matplotlib.ticker import FormatStrFormatter  
sns.set(font_scale=2)
import sys
import json
import os
import re
from scipy.stats import pearsonr
import pickle

## Util

In [16]:
dataset_name = ['ETTm1','ETTm2','ETTh1','ETTh2','Exchange','Weather', 'Traffic', 'Electricity','ILI', 'Solar Hour', 'Solar 10min', 'River Flow', 'Sunspot']
datasets = ['ettm1','ettm2','etth1','etth2','exchange_ltsf','weather_ltsf','illness_ltsf','sunspot', 'solar_hour', 'solar_10min','river_flow']
def replace_dataset_name(df):
    df['dataset'] = df['dataset'].replace('ettm1', 'ETTm1', regex=True) 
    df['dataset'] = df['dataset'].replace('ettm2', 'ETTm2', regex=True) 
    df['dataset'] = df['dataset'].replace('etth1', 'ETTh1', regex=True) 
    df['dataset'] = df['dataset'].replace('etth2', 'ETTh2', regex=True) 
    df['dataset'] = df['dataset'].replace('exchange_ltsf', 'Exchange', regex=True) 
    df['dataset'] = df['dataset'].replace('weather_ltsf', 'Weather', regex=True) 
    df['dataset'] = df['dataset'].replace('traffic_ltsf', 'Traffic', regex=True) 
    df['dataset'] = df['dataset'].replace('electricity_ltsf', 'Electricity', regex=True) 
    df['dataset'] = df['dataset'].replace('illness_ltsf', 'ILI', regex=True) 
    df['dataset'] = df['dataset'].replace('solar_hour', 'Solar Hour', regex=True) 
    df['dataset'] = df['dataset'].replace('solar_10min', 'Solar 10min', regex=True) 
    df['dataset'] = df['dataset'].replace('river_flow', 'River Flow', regex=True) 
    df['dataset'] = df['dataset'].replace('sunspot', 'Sunspot', regex=True) 
    return df

def sort_order(cat_res, sort_order_dict=None, inf_sort_order=False):
    # cat_res = cat_res.loc[cat_res['Infer hor.'] != '24']
    
    # inf_sort_order = ['48', '60', '96', '192', '336', '720', '1024']
    if inf_sort_order:
        inf_sort_order = ['24', '48', '96', '192','336', '720', '1024']
        cat_res['Infer hor.'] = pd.Categorical(cat_res['Infer hor.'], categories=inf_sort_order, ordered=True)
        cat_res = cat_res.sort_values(by='Infer hor.')
    
    if sort_order_dict is not None:
        for key, value in sort_order_dict.items():
            cat_res[key] = pd.Categorical(cat_res[key], categories=value, ordered=True)
            cat_res = cat_res.sort_values(by=key)
    
    return cat_res

def filed_filter(df, fileds_dict, eq=True):
    for key, values in fileds_dict.items():
        if eq:
            df = df[df[key].isin(values)]
        else:
            df = df[~df[key].isin(values)]
                
    return df

def add_spec_setting(res_dict, prefix=''):
    datasets = []
    infer_hor = []
    train_hor = []
    val_hor = []
    ND = []
    NRMSE = []
    seed = []
    for dataset in ['ETTm1','ETTm2','ETTh1','ETTh2','Exchange','Weather', 'Electricity','Traffic', 'Solar Hour', 'Solar 10min', 'River Flow', 'Sunspot']:
        for hor in ['96', '192', '336', '720']:
            try: 
                ND.append(res_dict[(res_dict['val_len']==hor) & (res_dict['Infer hor.']==hor) & (res_dict['Train hor.']==hor) & (res_dict['dataset']==dataset)]['ND'].values[0])
                NRMSE.append(res_dict[(res_dict['val_len']==hor) & (res_dict['Infer hor.']==hor) & (res_dict['Train hor.']==hor) & (res_dict['dataset']==dataset)]['NRMSE'].values[0])
                datasets.append(dataset)
                infer_hor.append(hor)
                train_hor.append(prefix+'spec')
                val_hor.append(prefix+'spec')
                seed.append(0)
            except:
                continue
            
    res_dict = {"dataset": datasets, "Infer hor.": infer_hor, "Train hor.": train_hor, "val_len": val_hor, "ND": ND , "NRMSE": NRMSE, "seeds": seed,}
    return pd.DataFrame(res_dict)

def plot_multi_marked_line(res,  x,  y, hue=None, style=None, title=None, datasets=['ETTh1'],save_name=None,line_styles=False, 
                           line_style_list=None,errorbar=None, palette=None, data_char=None, axvline=None,axhline=None, axhline_label=None,
                           hight=5, width=6, x_min=None, x_max=None, all_in_one=False, font_scale=1.5):
    sns.set(font_scale=font_scale)
    sub_num = len(datasets)
    if palette:
        sns.set_palette(palette)
    if sub_num > 1 and not all_in_one:
        plt.figure()
        total_len = sub_num * width
        fig, ax = plt.subplots(1, sub_num,figsize=(total_len, hight))  
        legend = False
        for i in range(sub_num):
            if line_style_list is not None:
                local_line_style = line_style_list[i]
            else:
                local_line_style = line_styles
                
            if datasets[i] == 'ETT':
                sub_res = res[(res['dataset']=='ETTm1') | (res['dataset']=='ETTm2') | (res['dataset']=='ETTh1') | (res['dataset']=='ETTh2')]
            else:
                sub_res = res[res['dataset']==datasets[i]]
                
            if i == (sub_num-1):
                legend = True
            sns.lineplot(
                data=sub_res, legend=legend,
                x=x, y=y, hue=hue, style=style, ax=ax[i], errorbar=errorbar,
                markers=True, dashes=local_line_style, linewidth=2,markersize=10
            )
            if axvline is not None:
                ax[i].axvline(x=axvline, color='r', linestyle='--', linewidth=2,markersize=10) 
            ax[i].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
            
            if data_char is not None:
                season = data_char[datasets[i]]['Seasonality']
                trend = data_char[datasets[i]]['Trend']
                data_char_str = f'\nFs:{season} Ft:{trend}'
            else:
                data_char_str = ''
            ax[i].set_title(f'{datasets[i]}{data_char_str}')  
            ax[i].tick_params(axis='y', labelsize=15)    
            ax[i].tick_params(axis='x', labelsize=16)    
            if i > 0:
                ax[i].set_ylabel('')
                
        if title is not None:
            fig.suptitle(title)
    else:
        plt.figure(figsize=(width, hight))
        if datasets[0] == 'ETT':
            sub_res = res[(res['dataset']=='ETTm1') | (res['dataset']=='ETTm2') | (res['dataset']=='ETTh1') | (res['dataset']=='ETTh2')]
        else:
            # sub_res = res[res['dataset']==datasets[0]]
            sub_res = res[res['dataset'].isin(datasets)]
        sns.lineplot(
            data=sub_res,
            x=x, 
            y=y, 
            hue=hue, style=style, errorbar=None,
            markers=True, dashes=line_styles, linewidth=2, markersize=10
        )
        if axvline is not None:
            plt.axvline(x=axvline, color='r', linestyle='--', linewidth=2,markersize=10) 
            
        if axhline is not None:
            plt.axhline(y=axhline, color='gray', linestyle='--', linewidth=2, label=axhline_label)
        
        if x_min is not None:
            plt.xlim(x_min, x_max)
            
        if title is not None:
            plt.title(title)
    
    # plt.legend(title=hue, loc='upper left', bbox_to_anchor=(1, 1))  
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1),title=hue)  
    plt.tight_layout() 
    if save_name is not None:
        plt.savefig(f'fig/{save_name}.svg', format='svg') 

## Load data

In [17]:
def load_result(root_dir, verbose=True, models=['PatchTST'], train_pred_len_list=['96', '192', '336', '720'], train_context_list=['96'], datasets=None):
    NRMSE = []
    ND = []
    CRPS = []
    train_pred_len_lists = []
    dataset_list = []
    seeds = []
    train_ctx = []
    model_list = []
    if datasets is None:
        datasets = ['ettm1','ettm2','etth1','etth2','exchange_ltsf','weather_ltsf','electricity_ltsf','traffic_ltsf','illness_ltsf']

    seed=0
    
    for dataset in datasets:
        for model in models:
            for train_pred_len in train_pred_len_list:
                for train_context in train_context_list:
                    csv_file = root_dir + f"{dataset}_{train_context}_{train_pred_len}_{model}_{seed}/test/metrics.csv"
                    try:
                        data = pd.read_csv(csv_file)
                    except:
                        if verbose:
                            print("cannot find ", csv_file) 
                        continue
                    
                    for idx in range(len(data)):
                        NRMSE.append(data['test_NRMSE'][idx])
                        ND.append(data['test_ND'][idx])
                        CRPS.append(data['test_CRPS'][idx])

                        dataset_list.append(dataset)
                        model_str = model
                        train_pred_len_lists.append(train_pred_len)
                        train_ctx.append(train_context)
                        model_list.append(model_str)
                        seeds.append(seed)

    res_dict = {"dataset": dataset_list, "pred hor.": train_pred_len_lists, "ND":ND , "NRMSE": NRMSE, 'CRPS':CRPS, "seeds": seeds,
                "model": model_list,'train_ctx': train_ctx}

    return pd.DataFrame(res_dict)


# ReVIN

## long

In [18]:
# PatchTST

datasets=None
root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_revin/norm_false/logs/"
# norm_false_res = load_result(root_dir=root_dir, verbose=False, models=['GRU_NVP', 'PatchTST'], datasets=datasets) 
norm_false_res = load_result(root_dir=root_dir, verbose=False, models=['PatchTST'], datasets=datasets) 
# norm_false_res['norm'] = 'None'
norm_false_res['scaling'] = 'false'
norm_false_res['revin'] = 'false'

root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_revin/norm_true/logs/"
norm_true_res = load_result(root_dir=root_dir, verbose=False, models=['PatchTST'], datasets=datasets) 
# norm_true_res['norm'] = 'ReVIN'
norm_true_res['scaling'] = 'false'
norm_true_res['revin'] = 'true'

root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_revin/norm_temp/logs/"
norm_temp_res = load_result(root_dir=root_dir, verbose=False, models=['PatchTST'], datasets=datasets) 
# norm_temp_res['norm'] = 'Scaling'
norm_temp_res['scaling'] = 'true'
norm_temp_res['revin'] = 'false'

patchtst_res= pd.concat([norm_false_res, norm_true_res, norm_temp_res], ignore_index=True)
patchtst_res['scaler'] = 'standard'

In [19]:
def get_df_results(root_path, models):
    combined_df = pd.DataFrame()
    datasets = ['etth1','exchange_ltsf','weather_ltsf','traffic_ltsf','electricity_ltsf']
    for scaler in ['identity', 'standard']:
        for revin in ['true', 'false']:
            for scaling in ['false', 'true']:
                norm_res = load_result(root_dir = f"{root_path}{scaler}_revin_{revin}_scaling_{scaling}/logs/", verbose=False, models=models, datasets=datasets)
                norm_res['scaler'] = scaler
                norm_res['revin'] = revin
                norm_res['scaling'] = scaling
                combined_df = pd.concat([combined_df, norm_res], ignore_index=True)
    return combined_df

In [20]:
gru_nvp_res = get_df_results('/data/Blob_WestJP/v-jiawezhang/log/abl_norm/', ['GRU_NVP'])
csdi_res = get_df_results('/data/Blob_EastUS/v-jiawezhang/log/abl_norm/', ['CSDI'])
timegrad_res = get_df_results('/data/Blob_EastUS/v-jiawezhang/log/abl_norm/', ['TimeGrad'])

In [21]:
combined_df = pd.concat([patchtst_res, gru_nvp_res, csdi_res, timegrad_res], ignore_index=True)

In [22]:
def reformat(res_dict):
    datasets = []
    pred_hor = []
    norm = []
    ND = []
    CRPS = []
    seed = []
    global_norm = []
    model = []
    for index, row in res_dict.iterrows():
        if row['revin'] == 'true' and row['scaling'] == 'false':
            norm.append('ReVIN')
        elif row['revin'] == 'false' and row['scaling'] == 'true':
            norm.append('Scaling')
        elif row['revin'] == 'false' and row['scaling'] == 'false':
            norm.append('None')
        else:
            continue
        
        datasets.append(row['dataset'])
        pred_hor.append(row['pred hor.'])
        ND.append(row['ND'])
        global_norm.append(row['scaler'])
        CRPS.append(row['CRPS'])
        model.append(row['model'])
        
            
    res_dict = {'model': model, "dataset": datasets, "gobal norm": global_norm, "pred hor.": pred_hor, "CRPS": CRPS, "ND": ND , "norm": norm}
    return pd.DataFrame(res_dict)

In [23]:
norm_res = reformat(combined_df)

In [24]:
norm_res = replace_dataset_name(norm_res)

sort_order_dict = {'pred hor.': ['96','192','336','720'], 'model': ['PatchTST', 'CSDI','TimeGrad', 'GRU_NVP'], 'norm': ['ReVIN', 'Scaling','None']}
sort_norm_res = sort_order(norm_res, sort_order_dict=sort_order_dict)

In [25]:
# sort_norm_res['CRPS/ND'] = sort_norm_res.apply(lambda x: f"{x['CRPS']:.3f}/{x['ND']:.3f}", axis=1)
sort_norm_res['ND'] = sort_norm_res.apply(lambda x: f"{x['ND']:.4f}", axis=1)
sort_norm_res['CRPS'] = sort_norm_res.apply(lambda x: f"{x['CRPS']:.4f}", axis=1)

sort_norm_res = sort_norm_res[sort_norm_res["gobal norm"]=='standard']

In [26]:
# Create a pivot table
pivot_table = pd.pivot_table(
    sort_norm_res,
    values=['CRPS'],
    index=['dataset', 'pred hor.'],
    columns=['model', 'norm'],
    aggfunc=lambda x: x.iloc[0],
    observed=False
)

def to_markdown(pivot_table):
    pivot_table.columns = ['_'.join(col).strip() for col in pivot_table.columns.values]
    pivot_table = pivot_table.reset_index()
    pivot_table_formatted_md = pivot_table.to_markdown()
    return pivot_table_formatted_md

In [28]:
# print(pivot_table_formatted_md)
pivot_table

CRPS                                                                                         
model                 PatchTST                    CSDI                 TimeGrad                 GRU_NVP                
norm                     ReVIN Scaling    None   ReVIN Scaling    None    ReVIN Scaling    None   ReVIN Scaling    None
dataset     pred hor.                                                                                                  
ETTh1       96          0.3212  0.3437  0.3447  0.2764  0.3630  0.3671   0.2958  0.6963  0.5434  0.2771  0.5946  0.4444
            192         0.3562  0.3943  0.3785  0.3553  0.4352  0.4673   0.3119  0.6410  0.5698  0.3076  0.4729  0.5056
            336         0.3737  0.4213  0.4099  0.3614  0.4192  0.4814   0.3950  0.6586  0.6192  0.3081  0.6240  0.4803
            720         0.3909  0.4881  0.4648  0.3960  0.4283  0.5609   0.3435  0.8670  0.5837  0.3641  0.7295  0.5862
Electricity 96          0.0857  0.0867  0.0848  0.0761  0.0735  0.0735   0.0771  0.0904  0.0961  0.0805  0.0882  0.0923
            192         0.0912  0.0918  0.0910  0.2749  0.2584  0.2475   0.0806  0.0936  0.0980  0.0833  0.0945  0.0935
            336         0.1001  0.1005  0.1006     NaN  0.2835  0.3136   0.0905  0.1002  0.1125  0.0925  0.0932  0.0982
            720         0.1160  0.1187  0.1178     NaN  0.2945  0.2667   0.1164  0.0957     NaN  0.1107  0.1036  0.1065
Exchange    96          0.0235  0.0254  0.0299  0.0216  0.0210  0.0343   0.0279  0.0478  0.0837  0.0246  0.0559  0.0605
            192         0.0336  0.0365  0.0404  0.0383  0.0388  0.0418   0.0364  0.0743  0.0638  0.0340  0.0814  0.0860
            336         0.0462  0.0532  0.0597  0.0508  0.0485  0.0510   0.0510  0.1142  0.1030  0.0475  0.0948  0.0637
            720         0.0777  0.0810  0.0824  0.0881  0.1042  0.0964   0.0800  0.1428  0.1177  0.0644  0.1257  0.0724
Traffic     96          0.2553  0.2546  0.2526     NaN     NaN     NaN   0.2309  0.2033  0.2094  0.2021  0.2134  0.1952
            192         0.2479  0.2447  0.2483     NaN     NaN     NaN   0.2110  0.2091  0.2079  0.2110  0.2034  0.2005
            336         0.2492  0.2470  0.2489     NaN     NaN     NaN   0.2306  0.2091  0.2271  0.2220  0.2143  0.1983
            720         0.2572  0.2568  0.2577     NaN     NaN     NaN      NaN     NaN     NaN  0.2524  0.2222  0.2198
Weather     96          0.0837  0.1082  0.1051  0.1202  0.0575  0.0591   0.0737  0.2308  0.2939  0.0837  0.1546  0.1005
            192         0.0858  0.1171  0.1136  0.1599  0.0717  0.0720   0.0788  0.2495  0.3138  0.0866  0.1342  0.1177
            336         0.0903  0.1167  0.1118  0.4296  0.0856  0.0977   0.0860  0.2646  0.2249  0.0868  0.1715  0.1145
            720         0.0953  0.1234  0.1117  0.1785  0.1419  0.0975   0.0941  0.1973  0.2069  0.0925  0.1146  0.1979

In [14]:
latex_code = pivot_table.to_latex(multicolumn=True, multirow=True)

In [15]:
print(latex_code)

\begin{tabular}{llllllllllllll}
\toprule
 &  & \multicolumn{12}{r}{CRPS} \\
 & model & \multicolumn{3}{r}{PatchTST} & \multicolumn{3}{r}{CSDI} & \multicolumn{3}{r}{TimeGrad} & \multicolumn{3}{r}{GRU_NVP} \\
 & norm & ReVIN & Scaling & None & ReVIN & Scaling & None & ReVIN & Scaling & None & ReVIN & Scaling & None \\
dataset & pred hor. &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{4}{*}{ETTh1} & 96 & 0.3212 & 0.3437 & 0.3447 & 0.2764 & 0.3630 & 0.3671 & 0.2958 & 0.6963 & 0.5434 & 0.2771 & 0.5946 & 0.4444 \\
 & 192 & 0.3562 & 0.3943 & 0.3785 & 0.3553 & 0.4352 & 0.4673 & 0.3119 & 0.6410 & 0.5698 & 0.3076 & 0.4729 & 0.5056 \\
 & 336 & 0.3737 & 0.4213 & 0.4099 & 0.3614 & 0.4192 & 0.4814 & 0.3950 & 0.6586 & 0.6192 & 0.3081 & 0.6240 & 0.4803 \\
 & 720 & 0.3909 & 0.4881 & 0.4648 & 0.3960 & 0.4283 & 0.5609 & 0.3435 & 0.8670 & 0.5837 & 0.3641 & 0.7295 & 0.5862 \\
\cline{1-14}
\multirow[t]{4}{*}{Electricity} & 96 & 0.0857 & 0.0867 & 0.0848 & 0.0761 & 0.0735 & 0.0735 & 0.0771 & NaN

## short

In [15]:
datasets=['electricity_nips', 'solar_nips', 'exchange_rate_nips','traffic_nips', 'wiki2000_nips']
train_pred_len_list = ['24', '30']
train_context_list = ['24', '30']
root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_revin/norm_false/logs/"
norm_false_res = load_result(root_dir=root_dir, verbose=False, models=['GRU_NVP', 'PatchTST'], datasets=datasets, train_pred_len_list=train_pred_len_list, train_context_list=train_context_list) 
norm_false_res['norm'] = 'None'

root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_revin/norm_true/logs/"
norm_true_res = load_result(root_dir=root_dir, verbose=False, models=['GRU_NVP', 'PatchTST'], datasets=datasets, train_pred_len_list=train_pred_len_list, train_context_list=train_context_list) 
norm_true_res['norm'] = 'ReVIN'

root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_revin/norm_temp/logs/"
norm_temp_res = load_result(root_dir=root_dir, verbose=False, models=['GRU_NVP', 'PatchTST'], datasets=datasets, train_pred_len_list=train_pred_len_list, train_context_list=train_context_list) 
norm_temp_res['norm'] = 'Scaling'

norm_res= pd.concat([norm_false_res, norm_true_res, norm_temp_res], ignore_index=True)

In [13]:
combined_df = pd.DataFrame()
datasets=['electricity_nips', 'solar_nips', 'exchange_rate_nips','traffic_nips', 'wiki2000_nips']
train_pred_len_list = ['24', '30']
train_context_list = ['24', '30']
for scaler in ['identity', 'standard']:
    for revin in ['true', 'false']:
        for scaling in ['false', 'true']:
            norm_res = load_result(root_dir = f"/data/Blob_WestJP/v-jiawezhang/log/abl_norm/{scaler}_revin_{revin}_scaling_{scaling}/logs/", verbose=False, models=['CSDI'], datasets=datasets, train_pred_len_list=train_pred_len_list, train_context_list=train_context_list) 
            norm_res['scaler'] = scaler
            norm_res['revin'] = revin
            norm_res['scaling'] = scaling
            combined_df = pd.concat([combined_df, norm_res], ignore_index=True)
            
combined_df = reformat(combined_df)

In [14]:
norm_res = replace_dataset_name(combined_df)
norm_res['ND'] = norm_res.apply(lambda x: f"{x['ND']:.4f}", axis=1)
norm_res['CRPS'] = norm_res.apply(lambda x: f"{x['CRPS']:.4f}", axis=1)
sort_norm_res = sort_norm_res[sort_norm_res["gobal norm"]=='identity']

In [17]:
pivot_table = pd.pivot_table(
    norm_res,
    values=['ND'],
    index=['dataset'],
    # columns=['model', 'norm'],
    columns=['norm'],
    aggfunc=lambda x: x.iloc[0]
)
pivot_table.columns = ['_'.join(col).strip() for col in pivot_table.columns.values]
pivot_table = pivot_table.reset_index()
pivot_table_formatted_md = pivot_table.to_markdown()

In [18]:
print(pivot_table_formatted_md)

|    | dataset            |   ND_None |   ND_ReVIN |   ND_Scaling |
|---:|:-------------------|----------:|-----------:|-------------:|
|  0 | electricity_nips   |  nan      |   nan      |       0.0657 |
|  1 | exchange_rate_nips |    0.0099 |     0.0109 |       0.0109 |
|  2 | solar_nips         |    1      |     0.5847 |       0.5832 |
|  3 | traffic_nips       |  nan      |     0.1731 |       0.1806 |


# Channel

In [9]:
datasets=None
root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_channel/ch_dep_false/logs/"
indep_res = load_result(root_dir=root_dir, verbose=False, models=['DLinear', 'PatchTST'], datasets=datasets) 
indep_res['channel'] = 'indep'

root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_channel/ch_dep_true/logs/"
dep_res = load_result(root_dir=root_dir, verbose=False, models=['DLinear', 'PatchTST'], datasets=datasets) 
dep_res['channel'] = 'dep'

In [10]:
chennel_res= pd.concat([dep_res,indep_res], ignore_index=True)
chennel_res = replace_dataset_name(chennel_res)
sort_order_dict = {'pred hor.': ['96','192','336','720']}
sort_chennel_res = sort_order(chennel_res, sort_order_dict=sort_order_dict)

sort_chennel_res['ND'] = sort_chennel_res.apply(lambda x: f"{x['ND']:.4f}", axis=1)
sort_chennel_res['CRPS'] = sort_chennel_res.apply(lambda x: f"{x['CRPS']:.4f}", axis=1)

In [11]:
# Create a pivot table
pivot_table = pd.pivot_table(
    sort_chennel_res,
    values=['CRPS'],
    index=['dataset', 'pred hor.'],
    columns=['model', 'channel'],
    aggfunc=lambda x: x.iloc[0]
)
pivot_table.columns = ['_'.join(col).strip() for col in pivot_table.columns.values]
pivot_table = pivot_table.reset_index()
pivot_table_formatted_md = pivot_table.to_markdown()

/tmp/ipykernel_3845555/372056143.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_table = pd.pivot_table(


In [12]:
print(pivot_table_formatted_md)

|    | dataset     |   pred hor. |   CRPS_DLinear_dep |   CRPS_DLinear_indep |   CRPS_PatchTST_dep |   CRPS_PatchTST_indep |
|---:|:------------|------------:|-------------------:|---------------------:|--------------------:|----------------------:|
|  0 | ETTh1       |          96 |             0.3334 |               0.329  |              0.3239 |                0.3212 |
|  1 | ETTh1       |         192 |             0.3611 |               0.4096 |              0.3609 |                0.3562 |
|  2 | ETTh1       |         336 |             0.3918 |               0.4177 |              0.3763 |                0.3737 |
|  3 | ETTh1       |         720 |             0.4266 |               0.4674 |              0.3882 |                0.3909 |
|  4 | ETTh2       |          96 |             0.3127 |               0.2034 |              0.1731 |                0.1746 |
|  5 | ETTh2       |         192 |             0.3351 |               0.2701 |              0.2001 |                0.2011 |


## short

In [33]:
datasets=['electricity_nips', 'solar_nips', 'exchange_rate_nips','traffic_nips','wiki2000_nips']
train_pred_len_list = ['24', '30']
train_context_list = ['24', '30']

root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_channel/ch_dep_false/logs/"
indep_res = load_result(root_dir=root_dir, verbose=False, models=['DLinear', 'PatchTST'], datasets=datasets) 
indep_res['channel'] = 'indep'

root_dir = "/data/Blob_WestJP/v-jiawezhang/log/abl_channel/ch_dep_true/logs/"
dep_res = load_result(root_dir=root_dir, verbose=False, models=['DLinear', 'PatchTST'], datasets=datasets) 
dep_res['channel'] = 'dep'

In [ ]:
chennel_res= pd.concat([dep_res,indep_res], ignore_index=True)
chennel_res = replace_dataset_name(chennel_res)

sort_chennel_res['ND'] = sort_chennel_res.apply(lambda x: f"{x['ND']:.4f}", axis=1)
sort_chennel_res['CRPS'] = sort_chennel_res.apply(lambda x: f"{x['CRPS']:.4f}", axis=1)

# Missing baselines - LSTM

In [33]:
train_pred_len_list=['96', '192', '336', '720', '24','36','48','60']
train_context_list=['96', '36']

datasets=['ettm1','etth1','exchange_ltsf','electricity_ltsf','traffic_ltsf','illness_ltsf']
root_dir = "/data/Blob_WestJP/v-jiawezhang/log/nn_baseline/logs/"
gru_res1 = load_result(root_dir=root_dir, verbose=False, models=['GRUForecaster'], datasets=datasets, train_pred_len_list=train_pred_len_list, train_context_list=train_context_list) 
gru_res1 = replace_dataset_name(gru_res1)

datasets=['ettm2','etth2','weather_ltsf']
root_dir = "/data/Blob_WestJP/v-jiawezhang/log/nn_baseline_scale/logs/"
gru_res2 = load_result(root_dir=root_dir, verbose=False, models=['GRUForecaster'], datasets=datasets, train_pred_len_list=train_pred_len_list, train_context_list=train_context_list) 
gru_res2 = replace_dataset_name(gru_res2)

gru_res = pd.concat([gru_res1, gru_res2], ignore_index=True)

long_crps = pd.read_csv("./exp_res/long_crps.csv")
long_nmae = pd.read_csv("./exp_res/long_nmae.csv")

In [34]:
new_res = gru_res.rename(columns={'dataset': 'Model', 'pred hor.': 'pred_len'})
new_res['pred_len'] = new_res['pred_len'].astype(str)

new_res['ND'] = new_res.apply(lambda x: f"{x['ND']:.4f}", axis=1)
new_res['CRPS'] = new_res.apply(lambda x: f"{x['CRPS']:.4f}", axis=1)

In [35]:
df_merged_nmae = long_nmae.copy()
df_merged_nmae['pred_len'] = df_merged_nmae['pred_len'].astype(str)

df_merged_crps = long_crps.copy()
df_merged_crps['pred_len'] = df_merged_crps['pred_len'].astype(str)


df_merged_crps = df_merged_crps.set_index(['Model', 'pred_len'])
df_merged_nmae = df_merged_nmae.set_index(['Model', 'pred_len'])
new_res = new_res.set_index(['Model', 'pred_len'])

In [36]:
combine_nmae_res = pd.merge(df_merged_nmae, new_res['ND'], left_index=True, right_index=True)
combine_nmae_res = combine_nmae_res.rename(columns={'ND': 'GRU'}).reset_index()

combine_crps_res = pd.merge(df_merged_crps, new_res['CRPS'], left_index=True, right_index=True)
combine_crps_res = combine_crps_res.rename(columns={'CRPS': 'GRU'}).reset_index()

In [37]:
combine_nmae_res.to_csv("./exp_res/update_long_nmae.csv", index=False)
combine_crps_res.to_csv("./exp_res/update_long_crps.csv", index=False)

# Computation Cost

In [2]:
def load_summary(root_dir, verbose=True, models=['ElasTST'], train_pred_len_list=['96-192-336-720'],train_context_list=['96'],datasets=None):
    dataset_list = []
    param_list = []
    mem_list = []
    train_time_list = []
    model_list = []
    if datasets is None:
        datasets = ['ettm1','ettm2','etth1','etth2','exchange_ltsf','weather_ltsf','electricity_ltsf','traffic_ltsf','illness_ltsf']

    seed=0
    
    for dataset in datasets:
        for model in models:
            for train_pred_len in train_pred_len_list:
                for train_context in train_context_list:
                    json_file = root_dir + f"{dataset}_{train_context}_{train_pred_len}_{model}_{seed}/summary.json"
                    
                    try:
                        with open(json_file, 'r') as file:
                            data = json.load(file)
                    except:
                        if verbose:
                            print("cannot find ", json_file) 
                        continue
                    
                    train_time = data['mean_train_batch_time']
                    trainable_parameters = data['trainable_parameters']
                    trainable_parameters_in_mb = trainable_parameters * 4 / (1024 ** 2)
                    train_mem_peak = data['memory_summary']['train']['mem_peak']
                    
                    dataset_list.append(dataset)
                    param_list.append(trainable_parameters_in_mb)
                    mem_list.append(train_mem_peak)
                    train_time_list.append(train_time)
                    model_list.append(model)



    res_dict = {"model":model_list, "dataset": dataset_list, 'NPARAMS (MB)': param_list, 'Max GPU Mem. (GB)': mem_list, "Avg. Train Time": train_time_list}

    return pd.DataFrame(res_dict)

def gpu_dataframe_to_markdown(df, filed='Max GPU Mem. (GB)'):
    # df.loc[:,'model'] = df['model'].replace('TransformerForecaster', 'Transformer', regex=True) 
    df.loc[:, filed] = df[filed].round(4)
    # df = df.pivot_table(index=['dataset'], columns='model', values=filed).reset_index()
    df = df.pivot_table(index=['dataset'], columns='model', values=filed, observed=False).reset_index()
    return df.to_markdown(index=False)

In [3]:
pred_len_list = ['96']
datasets = ['ettm1','electricity_ltsf', 'exchange_ltsf','weather_ltsf','traffic_ltsf']
root_dir = f"/data/Blob_WestJP/v-jiawezhang/log/comp_budget/logs/"
comp_budget = load_summary(root_dir, verbose=True, models=['PatchTST','DLinear','TimeGrad','GRU_NVP','CSDI'], train_pred_len_list=pred_len_list, datasets=datasets)

cannot find  /data/Blob_WestJP/v-jiawezhang/log/comp_budget/logs/electricity_ltsf_96_96_DLinear_0/summary.json
cannot find  /data/Blob_WestJP/v-jiawezhang/log/comp_budget/logs/weather_ltsf_96_96_CSDI_0/summary.json
cannot find  /data/Blob_WestJP/v-jiawezhang/log/comp_budget/logs/traffic_ltsf_96_96_CSDI_0/summary.json


In [4]:
comp_budget

,model,dataset,NPARAMS (MB),Max GPU Mem. (GB),Avg. Train Time
0,PatchTST,ettm1,5.141750,0.032442,0.029116
1,DLinear,ettm1,0.497341,0.018391,0.016629
2,TimeGrad,ettm1,1.133266,0.028333,0.039893
3,GRU_NVP,ettm1,1.008083,0.025000,0.032739
4,CSDI,ettm1,1.640350,0.147443,0.046746
5,PatchTST,electricity_ltsf,2.047001,0.047467,0.024961
6,TimeGrad,electricity_ltsf,3.310772,0.231849,0.036542
7,GRU_NVP,electricity_ltsf,3.509815,0.140491,0.021862
8,CSDI,electricity_ltsf,1.306679,0.286550,0.050155
9,PatchTST,exchange_ltsf,0.630341,0.018975,0.029638


In [5]:
# gpu_df = replace_dataset_name(combined_gpu_df)
gpu_markdown = gpu_dataframe_to_markdown(comp_budget, filed='Avg. Train Time')

In [6]:
print(gpu_markdown)

| dataset          |     CSDI |   DLinear |   GRU_NVP |   PatchTST |   TimeGrad |
|:-----------------|---------:|----------:|----------:|-----------:|-----------:|
| electricity_ltsf |   0.0502 |  nan      |    0.0219 |     0.025  |     0.0365 |
| ettm1            |   0.0467 |    0.0166 |    0.0327 |     0.0291 |     0.0399 |
| exchange_ltsf    |   0.0482 |    0.0171 |    0.0244 |     0.0296 |     0.0387 |
| traffic_ltsf     | nan      |    0.01   |    0.0358 |     0.0259 |     0.0367 |
| weather_ltsf     | nan      |    0.0094 |    0.0354 |     0.0259 |     0.0427 |
